## Süntaksianalüüs

Varasemalt leiti 50 raadiosaate suurusel alamhulgal süntaksianalüüsi abil hulk nelja teemasse jagunevaid märksõnu. Selles notebookis lisatakse süntaksianalüüsi kiht kõigile lausetele, millele on PhraseTaggeriga märgendatud mõni huvipakkuv nimeüksus.

In [14]:
from estnltk import Text
from estnltk.taggers import StanzaSyntaxEnsembleWebTagger
from estnltk_neural.taggers.syntax.stanza_tagger.ensemble_tagger import StanzaSyntaxEnsembleTagger
from estnltk_neural.taggers import StanzaSyntaxTagger
from estnltk.taggers import SyntaxDependencyRetagger
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
import csv
from collections import Counter

### I Tagger

Siin võetakse märgendamisel kasutusele Stanza ensemble taggeri asemel tavaline Stanza tagger, sest andmehulk on suurem ja taggeri töökiirus on parem.

In [3]:
stanza_tagger = StanzaSyntaxTagger(input_type='morph_analysis', input_morph_layer='morph_analysis', add_parent_and_children=True)

### II Andmete sisselugemine ja süntaksianalüüsi kihi peale märgendamine

Kõigepealt loetakse andmebaasist sisse laused, millele on PhraseTaggeriga märgendatud peale mõni huvipakkuvatest nimeüksustest.

In [15]:
sentences = []
sentence_ids = []

In [16]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [17]:
# ID asemel võtta tabelist sentence_ID
for row in cur.execute("SELECT sentence_ID, sentence FROM phrase_tagger_sentences"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    sentences.append(sentence)
    sentence_ids.append(sentence_id)

In [18]:
con.close()

In [19]:
len(sentences)

18569

Seejärel märgendatakse lausetele süntaksianalüüsi kiht. Ühtlasi eraldatakse laused, mis on taggeri jaoks liiga pikad (need on transkriptsioonifailid, kus oli puudu lauselõpumärkidest).

In [20]:
sentences_untagged = []

In [21]:
for sentence in sentences:
    try:
        stanza_tagger.tag(sentence)
    except:
        sentences_untagged.append(sentence)
        continue

In [23]:
len(sentences_untagged)

2

Esimene sellistest liiga pikkadest "lausetest":

In [26]:
for sentence in sentences_untagged:
    print(sentence.text)
    break

Tere siin Kuku raadios enam Keskpäevatund Tallinnas Marek Strandberg Priit Hõbemägi Tartus on Hannes Astok ja kurgid sulle raha mulle vastupidi kuidas keegi
siis see teema ja viib meid selle juurde et nagu kirjutada et riik tellib toidu väiksemat maksukoormat üles on kerkinud selline diskussiooni teema
et võiks võiks ju vähendada toidu käibemaksu toiduainetelt ettepanek mis on välja käidud on siis vist oli üheksa protsenti senise kahekümne asemel on nüüd diskussioon mis on
sel teemal areneb teede põhiliselt põhiliselt selles ühe punkti ümber et kas siis kui käibemaksu alandatakse kas toiduainete hinnad vähenevad või toiduainete hinnad ei vähene see on üks alateema on kindlasti see et et kas toiduainete hinnad Eestis on
on kõrgemad kui mujal Euroopas mujal maailmas või on nad madalamad aga see selle nimel juba
toiduainete hind on küsimus ju sellest millist hinda inimesed taluvad ja selles mõttes nii-öelda käib
maksuga mis on täna oluliselt kõrgem kui see mainitud üheksa protsenti on ini

### III Süntaksianalüüsi kihiga lausete salvestamine

Laused, millele õnnestus märgendada peale süntaksianalüüsi kiht salvestatakse andmebaasis:

In [27]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [28]:
cur.execute("CREATE TABLE sentences_syntax_analysis(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, sentence TEXT)")

In [30]:
for i in range(len(sentences)):
    if sentences[i] not in sentences_untagged:
        sentence_json = text_to_json(sentences[i])
        cur.execute("""INSERT INTO sentences_syntax_analysis
                                (sentence_ID, sentence)
                                VALUES (?, ?);""", (sentence_ids[i], sentence_json))
    con.commit()

In [31]:
con.close()